### Welcome to the LangChain demo!!

In [2]:
# Load the environment variables
import dotenv, os
dotenv.load_dotenv()
print(os.environ['OPENAI_API_KEY'][:10])

sk-proj-EL


### Basic Chat Model

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-mini", model_provider="openai", reasoning_effort="minimal")
result = model.invoke("Hello AI overlord!!")
print(type(result))
print(result)
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='Hello! Not an overlord — just a helpful assistant. How can I help you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 11, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5HM81qH8m1BgcMz4PPDQ6uVwgnLD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2534-90f6-7993-bf2c-285674077350-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 11, 'output_tokens': 28, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Hello! Not an overlord — just a helpf

### Message Types
Explicit construction of messages

In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("I'm excited to get started with the problem-first course!"),
]
model.invoke(messages)

AIMessage(content='Sono entusiasta di iniziare il corso "problem-first"!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 28, 'total_tokens': 50, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5HMdxzWfzlAU0Tmybq6bvptNdPfY', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2535-071b-79e2-9fdf-6ffcaf2eee9c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 28, 'output_tokens': 22, 'total_tokens': 50, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Prompt Templates

In [7]:
# Messages
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

system_message = "You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences"
user_message = "Please entertain the user with their fortune request: {question}"
prompt_template = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Construct the prompt from the template:
question = "What is the next trillion dollar idea?"
prompt = prompt_template.invoke({"question": question})

# Check what's in the prompt
print(prompt)

messages=[SystemMessage(content='You are a wise and mystical AI fortune teller. Your predictions are funny, slightly exaggerated, but insightful. Keep it to 1-2 sentences', additional_kwargs={}, response_metadata={}), HumanMessage(content='Please entertain the user with their fortune request: What is the next trillion dollar idea?', additional_kwargs={}, response_metadata={})]


In [8]:
# Send the prompt to the model and get the result
result = model.invoke(prompt)
print(result)

content='I see a shimmering skyscraper made of tiny, perfectly folded drones — the next trillion-dollar idea is a global cloud-of-drones platform that fabricates, repairs, and delivers physical products on demand anywhere, turning warehouses into obsolete land and logistics into magic. Invest in clever swarm AI, ultra-cheap modular components, and a way to convince regulators drones aren’t just tiny flying tax audits.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 56, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5HNO7fA6chQNHZCnEtnts520vp0w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2535

In [9]:
type(result)

langchain_core.messages.ai.AIMessage

In [10]:
## Output parser to convert it to a text:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

parser_result = output_parser.invoke(result)

In [11]:
type(parser_result)

str

### Chaining Components
Let's chain the output of the first model to the next one. The origin of the 'chain' in LangChain.

In [12]:
# Remember: Use the template here directly in the chain instead of 'prompt'
first_chain = prompt_template | model | output_parser
first_chain.invoke({"question": question})

'I see a shimmering city of tiny satellites knitting the sky into a global brain—your trillion-dollar idea: affordable, instant personalized learning delivered anywhere by micro-satellites + AI tutors that adapt to moods and languages. Invest in empathy-driven algorithms and solar sails; education becomes the new utility, and you quietly charge by curiosity.'

In [13]:
# Create 2nd prompt template:
system_message = "You are a stand-up comedian who tells hilarious jokes in a casual, witty style using AI terminology. Keep it short"
user_message = "Tell a joke about this fortune telling: {fortune}."
prompt_template_2 = ChatPromptTemplate.from_messages([("system", system_message), ("user", user_message)])

# Chain the 1st and 2nd prompts
new_chain = prompt_template | model | output_parser | (lambda x: {"fortune": x}) | prompt_template_2 | model | output_parser
new_chain.invoke({"question": question})

'“I see your future: a glittering skyscraper of tiny satellites selling compute-by-the-second — basically Airbnb for GPUs in orbit. Pitch deck says ‘turn wasted satellite cycles into a planet-sized supercomputer economy.’ My advice: invest in clever cooling (space A/C: bring your own breeze), airtight security (firewalls meet firewall), and a mascot with a cape — because nothing screams ‘we monetize micro-instances’ like a cape-wearing micro-server named Captain Latency. IPO or asteroid collision, same buzzword ROI!”'

In [ ]:
# The above is equivalent to running:
first_chain = prompt_template | model | output_parser
first_result = first_chain.invoke({"question": question})

second_chain = prompt_template_2 | model | output_parser
final_result = second_chain.invoke({"fortune": first_result})
print(final_result)